In [ ]:
#| default_exp mocking.functions

In [ ]:
#| export
import inspect
from pymoq.core import AnyCallable
from pymoq.signature_validators import SignatureValidator, signature_validator_from_arguments
from pymoq.return_value_generators import ReturnValueGenerator
from fastcore.basics import patch_to

In [ ]:
#| hide
from fastcore.test import test_fail
from nbdev.showdoc import show_doc

# Mocking functions
> From function object to function mock

## Checking arguments against original signature

For a given argument list, we want to know whether the original function can be called with that set of parameters. This can be done in two steps:
1. Extract the signature
2. Try to bind the argument list against the signature. This throws an exception if the arguments can not be matched against the signature

In [ ]:
def f(a: int, b: str, c:str|None =None) -> None:
    pass

Successful bind:

In [ ]:
inspect.signature(f).bind(1, "1", "2")

<BoundArguments (a=1, b='1', c='2')>

Unsuccessful bind:

In [ ]:
try:
    inspect.signature(f).bind(1)
except Exception as e:
    print(e)

missing a required argument: 'b'


In [ ]:
#| export
class FunctionMock:
    "Mocks a function object based on its signature"
    def __init__(self, func: AnyCallable):
        self._func = func
        self._signature = inspect.signature(self._func)
        self._setups = []
        
    def arguments_valid(self, *args, **kwargs) -> bool:
        "Given an arbitrary argument list (both positional and keyword arguments), returns True if the mocked function could be called with those arguments"
        try:
            self._signature.bind(*args, **kwargs)
            return True
        except:
            return False

In [ ]:
mock = FunctionMock(f)

Successful binds:

In [ ]:
assert mock.arguments_valid(1, "1", "2")
assert mock.arguments_valid(1, "1")
assert mock.arguments_valid(1, "1", c="2")
assert mock.arguments_valid(a=1, b="1", c="2")

Note that the types are not checked with this:

In [ ]:
assert mock.arguments_valid(1, 1)

Unsuccessful binds might be:

In [ ]:
assert not mock.arguments_valid(1) # too few arguments
assert not mock.arguments_valid(1,2,3,4) # too many arguments
assert not mock.arguments_valid(1,2,d=3) # unknown argument name

## Setup

We want to be able to create call-setups on the function mock. A setup consists of a signature validation and a return value generator. When the mock is called with a list of arguments, we check this list against the signature validator. If the call matches, we call the return value generator to generate the return value.

In [ ]:
#| export
class Setup:
    "This class bundles a signature validator with a call-result-action"
    def __init__(self, signature_validator: SignatureValidator):
        self._signature_validator = signature_validator
        
    def is_valid(self, *args, **kwargs) -> bool:
        "Uses the underlying `SignatureValidator` to determine if the argument list is valid"
        return self._signature_validator.is_valid(*args, **kwargs)
        
    def returns(self, return_value_generator: ReturnValueGenerator) -> None:
        "Set the `ReturnValueGenerator` to be called when this setup is successfully called"
        self._return_value_generator = return_value_generator
        
    def get_return_value(self, *args, **kwargs):
        "Calls the underlying `ReturnValueGenerator` the get the return value for the exact argument list"
        return self._return_value_generator(*args, **kwargs)

In [ ]:
show_doc(Setup.returns)

---

### Setup.returns

>      Setup.returns (return_value_generator:pymoq.return_value_generators.Retur
>                     nValueGenerator)

Set the `ReturnValueGenerator` to be called when this setup is successfully called

In [ ]:
@patch_to(FunctionMock)
def setup(self, *args, **kwargs):
    # todo: actually implement this
    sig = signature_validator_from_arguments(args, kwargs)
    self._setups.append(Setup(sig))
    
    return self._setups[-1]

In [ ]:
mock = FunctionMock(f)

mock.setup().returns(lambda: 5)
assert mock._setups[0].get_return_value() == 5

In [ ]:
assert mock._setups[0].is_valid()
assert not mock._setups[0].is_valid(1)

# Build library

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()